In [7]:
import pandas as pd
import glob
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from config import *

In [8]:
#path to txt files
file_path_cases = output_path+'/cases/*multianno.txt'
#list all txt files
cases = glob.glob(file_path_cases)

file_path_controls = output_path+'/control/*multianno.txt'
control = glob.glob(file_path_controls)

In [21]:
categories = ['exonic', 'UTR3', 'UTR5']
final_results = pd.DataFrame(columns=['patient_id'] + categories + ['label'])

In [22]:
target_gene = 'NOD2'
for file in cases + control:
    df = pd.read_csv(file, sep="\t", low_memory=False)
    patient_id = file.split("/")[-1].replace(".hg19_multianno.txt", "")
    function_col = 'Func.refGene'
    genes_col = 'Gene.refGene'
    df_filtered = df[df[genes_col].str.contains(r'\b' + target_gene + r'\b', case=False)]
    variant_counts = df_filtered[function_col].value_counts().reindex(categories, fill_value=0)
    label = 1 if "cases" in file else 0
    result_row = pd.DataFrame([[patient_id] + list(variant_counts) + [label]], columns=['patient_id'] + categories + ['label'])
    final_results = pd.concat([final_results, result_row], ignore_index=True)
final_results = final_results.sample(frac=1, random_state=42).reset_index(drop=True)
final_results.to_csv(vcf_path + "/shuffled.csv", index=False)
print(final_results)

    patient_id exonic UTR3 UTR5 label
0         HC22      4    2    0     0
1         CD19      2    2    0     1
2         CD13      5    2    1     1
3         HC28      2    1    1     0
4          HC1      2    1    1     0
..         ...    ...  ...  ...   ...
106        HC5      4    1    1     0
107       CD22      6    2    1     1
108       HC35      4    2    1     0
109       CD56      3    2    0     1
110       HC44      2    2    0     0

[111 rows x 5 columns]
